In [1]:
import os 
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build
load_dotenv()

True

In [2]:
#importing dataset
data = pd.read_csv("../data/ytUK.csv")

In [8]:
# Function for fetching necessary data from youtube

def youtube_data(df):
    api_key = os.getenv('API_KEY')
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Initializing empty lists to store the results 
    subscriber_counts = []
    total_views = []
    video_counts = []
    
    # Looping through the DataFrame, extracting the channel Od and making API requests
    for index, row in df.iterrows():
        # Extract the channel ID after the '@'
        channel_info = row['Name']
        channel_id = channel_info.split('@')[1] if '@' in channel_info else None
        
        if channel_id:
            #Define the parameters for the API request
            request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=channel_id
            )
            try:
                response = request.execute() # Executing the API request
            
                #  Extracting information from the response
                if "items" in response and len(response["items"]) > 0:
                    channel_info = response["items"][0]

                    # Subscriber count, total views and video count
                    subscriber_count = channel_info["statistics"].get("subscriberCount", "Hidden")
                    total_view = channel_info["statistics"].get("viewCount", 0)
                    video_count = channel_info["statistics"].get("videoCount", 0)
                    
                    # Appending the extracted data to the lists
                    subscriber_counts.append(subscriber_count)
                    total_views.append(total_view)
                    video_counts.append(video_count)
                else:
                    print(f"Channel ID {channel_id}: No data found")
                    subscriber_counts.append(0)
                    total_views.append(0)
                    video_counts.append(0)
            except Exception as e:
                print(f"Error fetching Data for channel ID {channel_id}: {str(e)}")
                subscriber_counts.append(0)
                total_views.append(0)
                video_counts.append(0)
        else:
            # If channel ID is not found, append NA to lists
            print(f"No valid channel ID found for row {index}")
            subscriber_counts.append(0)
            total_views.append(0)
            video_counts.append(0)
            
    # Adding the extracted data to the DataFrame
    data['Subscribers'] = subscriber_counts
    data['TotalViews'] = total_views
    data['TotalVideos'] = video_counts
    
    # Saving the updated Dataframe in csv format
    df.to_csv('../data/channel_statistics.csv', index=False)
        

In [10]:
youtube_data(data)

Channel ID sidemen: No data found
Channel ID juliusdein: No data found
Channel ID liverpoolfc: No data found
Channel ID sirhcchris2010: No data found
Channel ID thedadlab: No data found
